## Computational Graph

텐서플로우는 계산(computation)과 상태(state)를 하나의 통합된 데이터 흐름 그래프(unified dataflow graph)로 표현한다.
그래프의 노드(node)는 연산(operation)을 표현하고, 엣지(edge)는 연산자 간에 전달되는 데이터(tensorflow에서는 tensor라고 부름)를 나타낸다.

텐서플로우의 그래프는 계산에 대한 표현이다. 실제로 계산을 수행하기 위해서는 그래프가 세션(Session)에 런칭(launch)되어야 한다. 세션은 GPU, CPU 등의 디바이스를 이용하여 그래프의 연산을 실행한다.

이 notebook에서는 일반적인 파이썬 코드로 표현한 계산과 텐서플로우로 표현한 계산의 차이점에 대한 "느낌적인 느낌"을 이해함으로써 텐서플로우의 계산 그래프에 대해 알아보겠다.

#### Notebook을 ipynb 파일이 위치한 경로에서 실행할 것!!!

In [1]:
import os
import numpy as np
import tensorflow as tf

print(os.getcwd())
# Notebook을 ipynb 파일이 위치한 경로에서 실행할 것!!!
from show_graph import *


/data/highkh/TF_Lec/tensorflow-lecture/highkh


### 상수

프로그램에서 상수는 수정될 수 없는 값이다. 여기에는 리터럴(literal)이나 불변 변수(constant variable) 등이 포함되는데, 파이썬에는 불변 변수가 존재하지 않으므로 여기서는 리터럴으로만 이야기 한다.

In [2]:
3       # 정수 리터럴 3을 출력
"abc"   # 문자열 리터럴 "abc"

print(3)
print("abc")

3
abc


상수를 함수로 생각해보면 3은 호출했을 때, 항상 3을 반환하는 함수, "abc"는 호출했을 때 항상 "abc"를 반환하는 함수로 표현할 수 있다. 

In [3]:
def three():
    return 3
# 또는
three = lambda: 3

def abc():
    return "abc"
# 또는
abc = lambda: "abc"

### Test

In [4]:
'''
    input None instead of empty
'''

def one(x=None):
    return 1

print(one)
print(one())
print(one('y'))
print(one(y))

<function one at 0x7f4e59883e18>
1
1


NameError: name 'y' is not defined

In [5]:
'''
    input var
'''

lambda1_one = lambda y: 1

print(lambda1_one)
print(lambda1_one('x'))
print(lambda1_one())


<function <lambda> at 0x7f4e59832950>
1


TypeError: <lambda>() missing 1 required positional argument: 'y'

In [6]:
'''
    input None
'''

lambda2_one = lambda y=None: 1

print(lambda2_one)
print(lambda2_one('x'))
print(lambda2_one())
print(lambda2_one(x))

<function <lambda> at 0x7f4e59832ae8>
1
1


NameError: name 'x' is not defined

동일한 패턴이므로 상수를 함수로 변환하는 constant라는 함수로 만들어보자.

In [7]:
def constant(c):
    return lambda: c

three = constant(3)
abc = constant("abc")

print(three)
print(abc)

<function constant.<locals>.<lambda> at 0x7f4e59832e18>
<function constant.<locals>.<lambda> at 0x7f4e59883bf8>


### Test

In [8]:
'''
    input None instead of var
'''

def cont(m=None):
    return lambda: m

abcd = cont("abcd")

print(cont('a'))
print(cont())
print(abcd)
print(abcd())

<function cont.<locals>.<lambda> at 0x7f4e59842158>
<function cont.<locals>.<lambda> at 0x7f4e59883c80>
<function cont.<locals>.<lambda> at 0x7f4e598420d0>
abcd


이제 3은 *3을 반환하는 함수*로 표현되었다. 따라서 three를 출력하면 3대신 알수 없는 문자열이 출력되며, 3을 출력하기 위해서는 three를 먼저 평가(evaluation)해야 한다. 이를 평가하는 eval이라는 함수를 만들어보자.

In [9]:
def eval(t):
    return t()

print(eval(three))
print(eval(abc))

3
abc


함수 버전에서 constant 함수는 호출되었을 때 3 또는 abc를 반환하는 **연산**을 만들었다. **연산의 결과**는 eval함수에 three, abc등의 변수를 전달하여 얻을 수 있었다. 

두 코드 비교...

In [10]:
# 상수 리터럴
3
"abc"
print('literal:', 3)
print('literal:', "abc")

# 함수로 표현
three = constant(3)
abc = constant("abc")
print('function:', eval(three))
print('functions:', eval(abc))


print(three)
print(abc)

literal: 3
literal: abc
function: 3
functions: abc
<function constant.<locals>.<lambda> at 0x7f4e59832a60>
<function constant.<locals>.<lambda> at 0x7f4e59832e18>


동일한 코드를 텐서플로우로 표현해보자.

In [11]:
tf.reset_default_graph()

tf_three = tf.constant(3)
tf_abc = tf.constant("abc")
print(tf_three)
print(tf_abc)

with tf.Session() as sess:
    print('tf:', sess.run(tf_three))
    print('tf:', sess.run(tf_abc))

Tensor("Const:0", shape=(), dtype=int32)
Tensor("Const_1:0", shape=(), dtype=string)
tf: 3
tf: b'abc'


### Test
http://www.geeksforgeeks.org/byte-objects-vs-string-python/

In [17]:
''' python3
    (encode)string >>> byte   : machine readable
    (decode)byte   >>> string : human readable
'''

# init string
string = 'zuminternet'

# init byte obj
byte = b'zuminternet'

# encode
encoding = string.encode('ASCII')

# decode
decoding = byte.decode('ASCII')

# check encoding
if (encoding == byte): print('Encoding success')
else: print('Encoding fail')

# check decoding
if (decoding == string): print('Decoding success')
else: print('Decoding fail')

Encoding success
Decoding success


함수 버전과 마찬가지로 tf.constant 함수는 3 또는 abc를 반환하는 연산을 만든다. 그리고 연산의 결과에 대한 핸들(Handle)을 반환한다. 연산의 결과는 세션의 run 함수에 이 핸들을 전달하여 얻을 수 있다.

### 사칙연산

#### 함수 표현

In [19]:
# input 값이 정해지기 전에는 def f()에 있는 print는 실행되지 않음

def add(t1, t2):
    # return lambda: eval(t1) + eval(t2)
    def f():
        print('add')
        return eval(t1) + eval(t2)
    return f

def sub(t1, t2):
    # return lambda: eval(t1) - eval(t2)
    def f():
        print('sub')
        return eval(t1) - eval(t2)
    return f

def mul(t1, t2):
    # return lambda: eval(t1) * eval(t2)
    def f():
        print('mul')
        return eval(t1) * eval(t2)
    return f

def div(t1, t2):
    # return lambda: eval(t1) / eval(t2)
    def f():
        print('div')
        return eval(t1) / eval(t2)
    return f

t1 = constant(1)
t2 = constant(2)
t3 = constant(3)
t4 = constant(4)

# t3 = t1 + t2      # TypeError: unsupported operand type(s) for +: 'function' and 'function'
r1 = add(t1, t2)
r2 = mul(t1, t2)
r3 = sub(r2, r1)

print('evaluation r3')
print(eval(r3))

print('evaluation r2')
print(eval(r2))


evaluation r3
sub
mul
add
-1
evaluation r2
mul
2


#### 텐서플로우 구현

In [20]:
tf.reset_default_graph()

t1 = tf.constant(1)
t2 = tf.constant(2)
t3 = tf.constant(3)
t4 = tf.constant(4)

# t3 = t1 + t2      # TypeError: unsupported operand type(s) for +: 'function' and 'function'
r1 = tf.add(t1, t2)
r2 = tf.mul(t1, t2)
r3 = tf.sub(r2, r1)

with tf.Session() as sess:
  print('evaluation')
  print(sess.run(r3))

show_graph(tf.get_default_graph().as_graph_def())

evaluation
-1


### 질문

다음 코드는 함수 또는 텐서플로우 표현으로 어떻게 구현될 수 있을까?

```
a = 3
b = 10
c = 20

if a < 5:
  print(b)
else:
  print(c)
```

### Ans  

그런데 이런 식으로 일일이 session run을 하면 효율성이 떨어짐
    1. 같은 함수 중복실행
    2. 중복된 곳에서 session run 마다 input값이 다르게 선언된 경우

In [24]:
a = tf.constant(3)
b = tf.constant(10)
c = tf.constant(20)

with tf.Session() as sess:
    if sess.run(a) < 5:
        print(sess.run(b))
    else:
        print(sess.run(c))

10


### Feeding / Placeholder

위 예제의 계산 그래프는 항상 동일한 결과를 출력한다. 계산 그래프를 실제로 유용하게 만들려면 입력값을 그래프에 전달할 수 있어야 한다.

위의 constant 연산은 항상 동일한 값을 반환하는 연산이었다. 이번엔 입력값을 그대로 출력하는 함수를 생각해보자.
입력 값을 전달할 수 있어야하므로 eval 함수도 약간 수정하겠다.

In [10]:
def eval(t, feed_dict=None):
    if feed_dict == None:
        return t()
    if t in feed_dict.keys():
        return t(feed_dict[t], feed_dict)
    return t(feed_dict)

def identity():
    return lambda x, _: x
placeholder = identity

t1 = placeholder()
t2 = placeholder()

print(eval(t1, feed_dict={t1: 1}))
print(eval(t2, feed_dict={t2: 2}))

1
2


동일한 코드를 텐서플로우로 구현해보자

In [11]:
tf.reset_default_graph()

t1 = tf.placeholder(dtype=tf.int32)
t2 = tf.placeholder(dtype=tf.int32)

with tf.Session() as sess:
    print(sess.run(t1, feed_dict={t1: 1}))
    print(sess.run(t2, feed_dict={t2: 2}))

1
2


조금복잡한(?) 아래의 수식을 그래프로 구성해보자.

> f(x) = 3 * x + 4

In [12]:
def constant(c):
    return lambda _: c

def add(t1, t2):
    return lambda feed_dict: eval(t1, feed_dict) + eval(t2, feed_dict)

def mul(t1, t2, feed_dict=None):
    return lambda feed_dict: eval(t1, feed_dict) * eval(t2, feed_dict)

c1 = constant(3)
c2 = constant(4)
x = placeholder()

y = add(mul(c1, x), c2)
print('function version')
print('----------------')
print(eval(y, feed_dict={x: 0}))
print(eval(y, feed_dict={x: 1}))

tf.reset_default_graph()

c1 = tf.constant(3)
c2 = tf.constant(4)
x = tf.placeholder(dtype=tf.int32)

y = tf.add(tf.mul(c1, x), c2)

with tf.Session() as sess:
    print('tensorflow version')
    print('------------------')
    print(sess.run(y, feed_dict={x: 0}))
    print(sess.run(y, feed_dict={x: 1}))

show_graph(tf.get_default_graph().as_graph_def())

function version
----------------
4
7
tensorflow version
------------------
4
7


In [13]:
tf.reset_default_graph()

c1 = tf.constant(3)
c2 = tf.constant(4)
c3 = tf.constant(4)

x = tf.placeholder(dtype=tf.int32)
y = tf.placeholder(dtype=tf.int32)

z = tf.add(tf.mul(c1, x), c2)
w = tf.add(tf.mul(c2, y), c3)

with tf.Session() as sess:
    print('tensorflow version')
    print('------------------')
    print(sess.run(z, feed_dict={x: 0}))
    print(sess.run(w, feed_dict={y: 1}))

show_graph(tf.get_default_graph().as_graph_def())


tensorflow version
------------------
4
8
